In [2]:
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from data_exploration import calculate_power_complex

## Merged Data

In [ ]:
data_merged = pd.read_csv("processed_data/all_sensors_data.csv", index_col=0, parse_dates=["timestamp"])
data_merged

In [ ]:
data_merged.describe()

In [ ]:
data_merged_grouped_new = data_merged.groupby(["sensor_id", "timestamp"], as_index=True).mean()

In [ ]:
data_merged_grouped_new

In [ ]:
data_merged_grouped_new["P_plus"].unstack(level=0)

In [ ]:
heatmap_data = data_merged_grouped_new["P_plus"].unstack(level=0)

In [ ]:
heatmap_data_selected = heatmap_data["2019-11-25": "2020-02-25"]
heatmap_data_selected

In [ ]:
fig = px.imshow(
    heatmap_data_selected.T,
    labels=dict(x="timestamp", y="P_Plus", color="P_Plus Value"),
    x=heatmap_data_selected.index,
    y=heatmap_data_selected.columns,
    color_continuous_scale='RdYlGn_r',
    aspect="auto",
    title="P_Plus by Sensor",
    text_auto=True
)

fig.show()

In [ ]:
fig.write_image("visualizations/heatmap_p_plus.png")

### Missing data distribution for merged data

In [ ]:
info_data = pd.read_csv("processed_data/all_sensors_info.csv", index_col=[0,1])
info_data

In [ ]:
missing_pct = info_data.xs('missing_percentage', level='metric')

fig = px.imshow(
    missing_pct,
    labels=dict(x="Variable", y="Sensor ID", color="Missing %"),
    x=missing_pct.columns,
    y=missing_pct.index,
    color_continuous_scale='RdYlGn_r',
    aspect="auto",
    title="Missing Data Percentage by Sensor",
    text_auto=True
)

fig.update_layout(
    xaxis_title="Measures",
    yaxis_title="Sensor ID",
    height=1600,
    width=800,
)

fig.show()
fig.write_html("visualizations/missing_data_heatmap.html")

In [ ]:
print("Number of sensors with 100% missing values")
missing_pct[missing_pct==100].count()

In [ ]:
print("Number of sensors with more than 50% missing values")
missing_pct[missing_pct>=50].count()

In [ ]:
missing_pct_long = missing_pct.T.stack().reset_index(name='missing_percentage')
missing_pct_long.columns = ['variable', 'sensor_id', 'missing_percentage']

fig = px.histogram(
    missing_pct_long,
    x='missing_percentage',
    nbins=100,
    title='Distribution of Missing Data Percentage Across All Sensors and Variables',
    labels={'missing_percentage': 'Missing Data Percentage', 'count': 'Number of Sensor-Variable Pairs'}
)

fig.show()
fig.write_html("visualizations/missing_data_distribution_total.html")

In [ ]:
fig = px.histogram(
    missing_pct_long[missing_pct_long.variable.isin(["P_plus", "P_minus", "Q_plus", "Q_minus"] )],
    x='missing_percentage',
    facet_col='variable',
    facet_col_wrap=2,
    nbins=50,
    title='Distribution of Missing Data Percentage by Variable - Powers',
    labels={'missing_percentage': 'Missing Data Percentage', 'count': 'Count'},
    height=600
)

fig.show()
fig.write_html("visualizations/missing_data_distribution_powers.html")

In [ ]:
fig = px.histogram(
    missing_pct_long[missing_pct_long.variable.isin(["V_1", "V_2", "V_3"] )],
    x='missing_percentage',
    facet_col='variable',
    facet_col_wrap=3,
    nbins=50,
    title='Distribution of Missing Data Percentage by Variable - Phase Voltages',
    labels={'missing_percentage': 'Missing Data Percentage', 'count': 'Count'},
    height=400
)

fig.show()
fig.write_html("visualizations/missing_data_distribution_voltages.html")

## Power Data

In [2]:
all_sensors_power = pd.read_csv("processed_data/all_sensors_power_data.csv", index_col=0)
all_sensors_power.index = pd.to_datetime(all_sensors_power.index)
all_sensors_power

,P_plus,P_minus,Q_plus,Q_minus,sensor_id
2019-09-05 04:30:00,0.34,0.0,0.0,0.08,1
2019-09-05 04:45:00,0.00,0.0,0.0,0.00,1
2019-09-05 05:00:00,0.00,0.0,0.0,0.00,1
2019-09-05 05:15:00,0.00,0.0,0.0,0.00,1
2019-09-05 05:30:00,0.00,0.0,0.0,0.00,1
...,...,...,...,...,...
2023-11-12 22:45:00,0.00,0.0,0.0,0.00,161
2023-11-12 23:00:00,0.00,0.0,0.0,0.00,161
2023-11-12 23:15:00,0.00,0.0,0.0,0.00,161
2023-11-12 23:30:00,0.00,0.0,0.0,0.00,161


In [3]:
all_sensors_power_grouped = all_sensors_power.set_index(['sensor_id'], append=True)
all_sensors_power_grouped

,,P_plus,P_minus,Q_plus,Q_minus
,sensor_id,,,,
2019-09-05 04:30:00,1,0.34,0.0,0.0,0.08
2019-09-05 04:45:00,1,0.00,0.0,0.0,0.00
2019-09-05 05:00:00,1,0.00,0.0,0.0,0.00
2019-09-05 05:15:00,1,0.00,0.0,0.0,0.00
2019-09-05 05:30:00,1,0.00,0.0,0.0,0.00
...,...,...,...,...,...
2023-11-12 22:45:00,161,0.00,0.0,0.0,0.00
2023-11-12 23:00:00,161,0.00,0.0,0.0,0.00
2023-11-12 23:15:00,161,0.00,0.0,0.0,0.00


### P Plus Original and Daily Data Plots

In [4]:
heatmap_data_p_plus = all_sensors_power_grouped["P_plus"].unstack(level=1)
heatmap_data_p_plus

sensor_id,1,2,3,4,5,6,7,8,9,10,...,151,152,153,154,155,156,158,159,160,161
2019-09-05 04:30:00,0.340,NaN,0.144,NaN,0.820,NaN,0.488,NaN,NaN,NaN,...,0.240,NaN,0.432,NaN,0.184,NaN,NaN,NaN,NaN,NaN
2019-09-05 04:45:00,0.000,NaN,0.156,NaN,0.512,NaN,0.448,NaN,NaN,NaN,...,0.132,NaN,0.444,NaN,0.248,NaN,NaN,NaN,NaN,NaN
2019-09-05 05:00:00,0.000,NaN,0.148,NaN,0.492,NaN,2.008,NaN,NaN,NaN,...,0.184,NaN,0.504,NaN,0.384,NaN,NaN,NaN,NaN,NaN
2019-09-05 05:15:00,0.000,NaN,0.148,NaN,0.360,NaN,3.072,NaN,NaN,NaN,...,0.248,NaN,0.644,NaN,0.408,NaN,NaN,NaN,NaN,NaN
2019-09-05 05:30:00,0.000,NaN,0.152,NaN,0.060,NaN,1.840,NaN,NaN,NaN,...,0.228,NaN,0.384,NaN,0.588,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-12 22:45:00,1.088,0.300,1.440,0.084,1.048,0.136,1.588,5.264,1.476,0.444,...,0.224,0.280,0.276,0.140,1.756,1.036,1.100,0.916,1.968,0.0
2023-11-12 23:00:00,1.080,0.240,1.392,0.020,0.324,0.140,1.612,1.848,1.480,0.496,...,0.188,0.124,0.280,0.184,0.724,1.080,0.628,0.932,2.092,0.0
2023-11-12 23:15:00,1.080,0.244,1.484,0.116,0.200,0.160,1.980,2.608,1.164,1.596,...,0.148,0.516,0.268,0.176,0.808,1.076,1.852,0.928,1.244,0.0
2023-11-12 23:30:00,1.080,0.204,1.404,0.264,0.128,0.284,1.296,3.732,1.192,3.372,...,0.216,0.120,0.416,0.100,1.120,1.028,2.836,0.988,1.216,0.0


In [6]:
heatmap_data_p_plus_daily = heatmap_data_p_plus.resample("D").mean()
heatmap_data_p_plus_daily

sensor_id,1,2,3,4,5,6,7,8,9,10,...,151,152,153,154,155,156,158,159,160,161
2019-09-05,0.399590,0.537143,0.230769,0.018743,0.270513,0.383257,1.023795,0.555200,NaN,0.266457,...,0.379077,0.581886,0.532308,0.135086,0.549026,0.003200,NaN,NaN,NaN,NaN
2019-09-06,0.601792,0.401875,0.241833,0.018500,0.303375,0.627250,0.733708,1.016583,NaN,0.359833,...,0.399083,0.733917,0.718375,0.239750,0.451042,0.003167,NaN,NaN,NaN,NaN
2019-09-07,0.626125,0.701792,0.344750,0.015458,0.709667,0.438125,0.725750,0.978000,NaN,0.580667,...,0.363792,0.748542,1.100125,0.223708,0.659292,0.011167,NaN,NaN,NaN,NaN
2019-09-08,0.653458,0.429542,0.236708,0.014708,0.451750,0.661750,1.210208,1.105542,NaN,0.467458,...,0.304875,0.568292,1.129417,0.199375,0.538875,0.003125,NaN,NaN,NaN,NaN
2019-09-09,0.598000,0.335000,0.291583,0.014500,0.372958,0.434500,0.917792,0.856833,NaN,0.370458,...,0.359292,0.688208,0.621875,0.254542,0.404833,0.003167,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-08,0.653458,0.279000,1.336667,0.054458,0.218208,0.724917,1.494708,2.311500,1.726917,0.472917,...,0.445250,0.324750,0.386208,0.174792,0.988500,0.683750,0.586917,0.926500,1.263458,0.016208
2023-11-09,0.651333,0.392583,1.229417,0.062083,0.316792,0.408083,1.801083,2.295417,1.815500,0.375375,...,0.440667,0.461708,0.302333,0.228583,1.021892,0.664875,0.545625,0.755917,1.239250,0.011125
2023-11-10,0.655708,0.345375,1.157708,0.081167,0.325875,0.438167,1.661625,2.615875,2.192500,0.403417,...,0.423167,0.460500,0.535625,0.310500,0.889417,0.664917,0.587917,0.818750,1.230667,0.006958
2023-11-11,0.654708,0.694167,1.341708,0.066792,0.344917,0.271000,1.340708,2.876417,1.596458,0.653958,...,0.422708,0.394458,0.776458,0.259333,1.081917,0.783750,0.621208,0.889625,1.259750,0.154625


In [ ]:
fig = px.imshow(
    heatmap_data_p_plus.T,
    labels=dict(x="timestamp", y="P_Plus", color="P_Plus Value"),
    x=heatmap_data_p_plus.index,
    y=heatmap_data_p_plus.columns,
    color_continuous_scale='RdYlGn_r',
    aspect="auto",
    title="P_Plus by Sensor",
    text_auto=True
)

fig.update_layout(
    xaxis_title="Timestamp",
    yaxis_title="Sensor ID",
    height=1600,
    width=2000,
)

# fig.write_html("visualizations/heatmap_p_plus.html", include_plotlyjs='cdn')

### All Power Data

In [6]:
units = {
    "P_plus": "kW",
    "P_minus": "kW",
    "Q_plus": "kVAR",
    "Q_minus": "kVAR",
    "P": "kW",
    "Q": "kVAR",
    "V_1": "V",
    "V_2": "V",
    "V_3": "V"
}

In [4]:
def plot_heatmap(
    metric: str, heatmap_data_metric: pd.DataFrame, is_daily=False, suffix: str = None
):
    if is_daily:
        heatmap_data_metric = heatmap_data_metric.resample("D").mean()

    title_suffix = "Daily Average " if is_daily else ""
    fig = px.imshow(
        heatmap_data_metric.T,
        labels=dict(
            x="timestamp",
            y="Sensor ID",
            color=f"{title_suffix}{metric} Value ({units.get(metric, '')})",
        ),
        x=heatmap_data_metric.index,
        y=heatmap_data_metric.columns,
        color_continuous_scale="RdYlGn_r",
        aspect="auto",
        title=f"{title_suffix}{metric} by Sensor",
        text_auto=True,
    )

    fig.update_layout(
        xaxis_title="Timestamp",
        yaxis_title="Sensor ID",
        height=1600,
        width=2000,
    )

    fig.write_html(
        f"visualizations/heatmap_{metric}{'_daily' if is_daily else ''}{f'_{suffix}' if suffix is not None else ''}.html",
        include_plotlyjs="cdn",
    )


def plot_table(metric: str, heatmap_data_metric: pd.DataFrame, suffix: str = None):
    description = heatmap_data_metric.describe(percentiles=[0.05, 0.95, 0.99])
    styled_description = (
        description.T.style.background_gradient(
            cmap="RdYlGn_r", axis=0
        )  # Column-wise gradient
        .set_caption(f"{metric} Statistics by Sensor")
        .format(precision=6)
        .set_properties(
            **{
                "text-align": "center",
                "font-size": "9pt",
                "font-family": "Arial, sans-serif",
                "border": "none",
                "padding": "6px",
            }
        )
        .set_table_styles(
            [
                {
                    "selector": "th",
                    "props": [
                        ("background-color", "white"),
                        ("color", "#333"),
                        ("font-weight", "bold"),
                        ("padding", "8px"),
                        ("text-align", "center"),
                        ("border", "none"),
                        ("font-size", "10pt"),
                        ("font-family", "Arial, sans-serif"),
                    ],
                },
                {
                    "selector": "th.index_name",  # Top-left corner cell
                    "props": [
                        ("background-color", "white"),
                        ("color", "#333"),
                        ("border", "none"),
                    ],
                },
                {
                    "selector": "th.row_heading",  # Row index (sensor_id column)
                    "props": [
                        ("background-color", "white"),
                        ("color", "#333"),
                        ("font-weight", "normal"),
                        ("border", "none"),
                        ("font-size", "9pt"),
                    ],
                },
                {
                    "selector": "td",
                    "props": [
                        ("padding", "6px"),
                        ("border", "none"),
                        ("font-family", "Arial, sans-serif"),
                    ],
                },
                {
                    "selector": "table",
                    "props": [
                        ("border-collapse", "collapse"),
                        ("margin", "20px auto"),
                        ("font-family", "Arial, sans-serif"),
                    ],
                },
                {
                    "selector": "caption",
                    "props": [
                        ("font-size", "14pt"),
                        ("font-weight", "bold"),
                        ("text-align", "center"),
                        ("padding", "10px"),
                        ("font-family", "Arial, sans-serif"),
                        ("color", "#333"),
                    ],
                },
            ]
        )
    )

    styled_description.to_html(
        f"visualizations/table_heatmap_{metric}{f'_{suffix}' if suffix is not None else ''}_description.html"
    )

In [9]:
for metric in ["P_plus", "P_minus", "Q_plus", "Q_minus"]:
    print(f"Metric: {metric}")
    heatmap_data_metric = all_sensors_power_grouped[metric].unstack(level=1)
    
    plot_heatmap(metric, heatmap_data_metric, is_daily=False)
    plot_heatmap(metric, heatmap_data_metric, is_daily=True)
    plot_table(metric, heatmap_data_metric)

Metric: P_plus
Metric: P_minus
Metric: Q_plus
Metric: Q_minus


In [7]:
df_complex_pwr = calculate_power_complex(all_sensors_power_grouped)
df_complex_pwr

,,P,Q
,sensor_id,,
2019-09-05 04:30:00,1,0.34,-0.08
2019-09-05 04:45:00,1,0.00,0.00
2019-09-05 05:00:00,1,0.00,0.00
2019-09-05 05:15:00,1,0.00,0.00
2019-09-05 05:30:00,1,0.00,0.00
...,...,...,...
2023-11-12 22:45:00,161,0.00,0.00
2023-11-12 23:00:00,161,0.00,0.00
2023-11-12 23:15:00,161,0.00,0.00


In [8]:
for metric in ["P", "Q"]:
    print(f"Metric: {metric}")
    heatmap_data_metric = df_complex_pwr[metric].unstack(level=1)

    plot_heatmap(metric, heatmap_data_metric, is_daily=False)
    plot_heatmap(metric, heatmap_data_metric, is_daily=True)
    plot_table(metric, heatmap_data_metric)

Metric: P
Metric: Q


### All Data Outliers Removed

In [9]:
all_sensors_power_outliers_rm = pd.read_csv("processed_data/all_sensors_power_data_outliers_removed.csv", index_col=0)
all_sensors_power_outliers_rm.index = pd.to_datetime(all_sensors_power_outliers_rm.index)

all_sensors_power_outliers_rm_grouped = all_sensors_power_outliers_rm.set_index(['sensor_id'], append=True)
all_sensors_power_outliers_rm_grouped

,,P_plus,P_minus,Q_plus,Q_minus
,sensor_id,,,,
2019-09-05 04:30:00,1,0.34,0.0,0.0,0.08
2019-09-05 04:45:00,1,0.00,0.0,0.0,0.00
2019-09-05 05:00:00,1,0.00,0.0,0.0,0.00
2019-09-05 05:15:00,1,0.00,0.0,0.0,0.00
2019-09-05 05:30:00,1,0.00,0.0,0.0,0.00
...,...,...,...,...,...
2023-11-12 22:45:00,161,0.00,0.0,0.0,0.00
2023-11-12 23:00:00,161,0.00,0.0,0.0,0.00
2023-11-12 23:15:00,161,0.00,0.0,0.0,0.00


In [10]:
for metric in ["P_plus", "P_minus", "Q_plus", "Q_minus"]:
    print(f"Metric: {metric}")
    heatmap_data_metric = all_sensors_power_outliers_rm_grouped[metric].unstack(level=1)

    plot_heatmap(metric, heatmap_data_metric, is_daily=False, suffix="outliers_rm")
    plot_heatmap(metric, heatmap_data_metric, is_daily=True, suffix="outliers_rm")
    plot_table(metric, heatmap_data_metric, suffix="outliers_rm")

Metric: P_plus
Metric: P_minus
Metric: Q_plus
Metric: Q_minus


In [11]:
df_complex_pwr_outliers_rm = calculate_power_complex(all_sensors_power_outliers_rm_grouped)
df_complex_pwr_outliers_rm
for metric in ["P", "Q"]:
    print(f"Metric: {metric}")
    heatmap_data_metric = df_complex_pwr_outliers_rm[metric].unstack(level=1)

    plot_heatmap(metric, heatmap_data_metric, is_daily=False, suffix="outliers_rm")
    plot_heatmap(metric, heatmap_data_metric, is_daily=True, suffix="outliers_rm")
    plot_table(metric, heatmap_data_metric, suffix="outliers_rm")

Metric: P
Metric: Q


## Voltage Data

In [10]:
all_sensors_voltage = pd.read_csv("processed_data/all_sensors_voltage_data.csv", index_col=0)
all_sensors_voltage.index = pd.to_datetime(all_sensors_voltage.index)
all_sensors_voltage

,V_1,sensor_id,V_2,V_3
2019-01-25 03:10:00,244.1,1,NaN,NaN
2019-01-25 03:20:00,243.9,1,NaN,NaN
2019-01-25 03:30:00,241.6,1,NaN,NaN
2019-01-25 03:40:00,243.1,1,NaN,NaN
2019-01-25 03:50:00,242.8,1,NaN,NaN
...,...,...,...,...
2023-11-07 23:10:00,232.7,161,225.0,232.7
2023-11-07 23:20:00,234.7,161,226.9,233.7
2023-11-07 23:30:00,235.9,161,229.3,235.2
2023-11-07 23:40:00,234.9,161,229.3,234.4


In [11]:
all_sensors_voltage_grouped = all_sensors_voltage.set_index(['sensor_id'], append=True)
all_sensors_voltage_grouped

,,V_1,V_2,V_3
,sensor_id,,,
2019-01-25 03:10:00,1,244.1,NaN,NaN
2019-01-25 03:20:00,1,243.9,NaN,NaN
2019-01-25 03:30:00,1,241.6,NaN,NaN
2019-01-25 03:40:00,1,243.1,NaN,NaN
2019-01-25 03:50:00,1,242.8,NaN,NaN
...,...,...,...,...
2023-11-07 23:10:00,161,232.7,225.0,232.7
2023-11-07 23:20:00,161,234.7,226.9,233.7
2023-11-07 23:30:00,161,235.9,229.3,235.2


In [12]:
for metric in ["V_1", "V_2", "V_3"]:
    print(f"Metric: {metric}")
    heatmap_data_metric = all_sensors_voltage_grouped[metric].unstack(level=1)

    plot_heatmap(metric, heatmap_data_metric, is_daily=False)
    plot_heatmap(metric, heatmap_data_metric, is_daily=True)
    plot_table(metric, heatmap_data_metric)

Metric: V_1
Metric: V_2
Metric: V_3


In [13]:
vlt_file_path = f"PhaseVoltages/2.csv"
# Read and process voltage data
data_vlt = pd.read_csv(
    vlt_file_path, parse_dates=["time"], date_format="%d-%b-%Y %H:%M:%S.%f"
)
data_vlt.rename(columns={"time": "timestamp"}, inplace=True)
data_vlt.drop(columns=["serialno"], inplace=True)

data_vlt = data_vlt.pivot_table(
    index="timestamp", columns="variable", values="value", dropna=False
)
data_vlt.columns.name = None
data_vlt.rename(columns={"V_L1": "V_1", "V_L2": "V_2", "V_L3": "V_3"}, inplace=True)
full_range_vlt = pd.date_range(
    start=data_vlt.index.min(), end=data_vlt.index.max(), freq="10min"
)
data_vlt = data_vlt.reindex(full_range_vlt)
data_vlt

,V_1,V_2,V_3
2019-01-25 03:10:00,245.1,238.5,234.4
2019-01-25 03:20:00,245.5,237.3,234.2
2019-01-25 03:30:00,243.0,236.5,229.6
2019-01-25 03:40:00,242.6,237.0,230.2
2019-01-25 03:50:00,241.5,232.6,233.4
...,...,...,...
2023-11-07 23:10:00,231.4,228.3,229.9
2023-11-07 23:20:00,231.0,233.7,229.0
2023-11-07 23:30:00,232.5,236.8,228.6
2023-11-07 23:40:00,237.5,238.0,226.9


In [14]:
data_vlt.describe()

,V_1,V_2,V_3
count,234292.000000,234292.000000,234221.000000
mean,235.887531,232.945629,227.423615
std,5.623409,5.301753,7.588285
min,0.000000,0.000000,0.000000
25%,232.300000,229.600000,222.600000
50%,235.300000,233.200000,227.500000
75%,239.300000,236.500000,232.200000
max,255.000000,250.000000,251.400000


In [15]:
data_vlt["V_2"].idxmin()

Timestamp('2021-05-09 17:50:00')

In [18]:
selected_range = data_vlt.loc["2021-05-09 17:00:00":"2021-05-09 18:50:00"]
fig = px.line(
    selected_range,
    x=selected_range.index,
    y=["V_1", "V_2", "V_3"],
    title="Phase Voltages Over Time for Sensor 2",
    labels={"value": "Voltage (V)", "timestamp": "Time"},
)
fig.show()

## Single Sensor

In [ ]:
# run data_exploration.py to get data_merged
# Insert data
i = 2  # Example sensor number
data_merged = pd.read_csv(f"processed_data/sensor_{i}_30min_merged.csv", index_col=0, parse_dates=True)
data_merged.head()

In [ ]:
fig = px.line(
    data_merged,
    x=data_merged.index,
    y=["P_plus", "P_minus", "Q_plus", "Q_minus"],
    title="Active and Reactive Power over Time",
)
fig.show()

In [ ]:
fig = px.line(
    data_merged,
    x=data_merged.index,
    y=["V_1", "V_2", "V_3"],
    title="3-Phase Voltage over Time",
)
fig.show()